# Le GIL

Le GIL ou [Global Interpreter Lock]() est un verrou unique auquel l'interpréteur Python fait appel constamment pour protéger tous les objets qu'il manipule contre des accès concurrentiels.

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Deux listes en parallèlle

On mesure le temps nécessaire pour créer deux liste et comparer ce temps avec celui que cela prendrait en parallèle.

In [1]:
def create_random_list(n):
    res = []
    for i in range(n):
        res.append(i)
    return res

%timeit create_random_list(100000)

9.41 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


En parallèle avec le module [asyncio](https://docs.python.org/3/library/asyncio.html) :

In [5]:
import asyncio
loop = asyncio.get_event_loop()

In [6]:
async def async_create_random_list(n):
    return create_random_list(n)

def run(loop, nb):
    loop.run_until_complete(async_create_random_list(nb))

nb = 100000

%timeit run(loop, nb)

8.89 ms ± 86.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Le temps est similaire pour une boucle. Avec deux maintenant :

In [9]:
def run2(loop, nb):
    loop.run_until_complete( asyncio.gather(
            async_create_random_list(nb),
            async_create_random_list(nb)))

nb = 100000

%timeit run2(loop, nb)

19.8 ms ± 824 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Tout se passe comme si le calcul n'était pas paralléliser. Et pourtant.

In [3]:
loop.close()